In [2]:
!pip install torch

In [1]:
#from transformers import pipeline
from optparse import OptionParser
#from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import sys

In [4]:
json_path="/Users/shawnschulz/Downloads/01_test2.json"

with open(json_path) as json_file:
    schema_dictionary = json.load(json_file)

In [ ]:
### Should probably update this to use a python class, so that when we add new node types
### we can reuse some methods without having to minor edits to make them compatible with
### new node types. This is okay for now tho, since that will require thinking some new logic for
### how the schema deals with sending outputs of nodes to nodes of different types

In [23]:
def schemaListToDictionary(schemaList):
    '''
        Takes nodes or edges list and makes a dictionary you can index list elements using the ID, probably
        only worth running this function if you have a really big graph you need to access nodes or edges of
        many times, but this option is available for that
    '''
    newDict={}
    for dictionary in schemaList:
        newDict[dictionary['id']] = dictionary.copy()
    return newDict

def hashedMappedSchemaDictionary(schema_dictionary):
    '''
        Take a whole schema dictionary and return a version that now has ID key'd nested dictionaries
        rather than lists
    '''
    newDict={'nodes':{}, 'edges':{}}
    nodesDict = schemaListToDictionary(schema_dictionary['nodes'])
    edgesDict = schemaListToDictionary(schema_dictionary['edges'])
    newDict['nodes'] = nodesDict
    newDict['edges'] = edgesDict
    return newDict

In [24]:
def findRoots(schema_dictionary):
    '''
        Finds the roots of a schema
        1. Get all the ids of all nodes, put in a stack
        2. Check what targets are currently in the schema
        3. Remove them as you find them

        Give schema dictionary (direct from json file), a dictionary
        Returns the stack of roots of the tree, False if schema nodes have no sources
    '''
    stack = []
    for node in schema_dictionary['nodes']:
        stack.append(node['id'])
    for edge in schema_dictionary['edges']:
        if edge['target'] in stack:
            stack.remove(edge['target'])
    return stack

In [25]:
def updateNodePrompts(node_prompt_dictionary, schema_dictionary):
    '''
        Takes a 
    '''
    new_dictionary = schema_dictionary.copy()
    for node in new_dictionary['nodes']:
        if node['id'] in node_prompt_dictionary.keys():
            old_prompt = node['data']['prompt'] 
            new_prompt = node_prompt_dictionary['id'] + ' \n' + old_prompt
            node['data']['prompt'] = new_prompt
    return(new_dictionary)
    
            

In [26]:
def removeNodeIDs(id_list, schema_dictionary):
    '''
        Takes a list of id's to be removed and returns a new schema dictionary with the node
        id's removed

        Note, when making graph smaller, it will usually make more sense to remove unique
        edge id's than nodes. However if there is some reason to this node removal function is here

        This is also really slow, consider changing if u actually make graphs really big
    '''
    new_dictionary = schema_dictionary.copy()
    for id in id_list:
        for node in new_dictionary['nodes']:
            if node['id'] == id:
                new_dictionary['nodes'].remove(node)
    return new_dictionary

In [27]:
def removeEdgeIDs(id_list, schema_dictionary):
    '''
        takes a list of edge id's and returns a new schema dictionary with the edge id's removed
        Note: important that you remove by edgeid, not source or target since the edge id is unique
    '''
    new_dictionary = schema_dictionary.copy()
    for id in id_list:
        for edge in new_dictionary['edges']:
            if edge['id'] == id:
                new_dictionary['edges'].remove(edge)
    return new_dictionary

In [28]:
#to start with, I think we should always start from the first node created. May make sense in the future to allow user to specify
#what node they want to start with, but i am too lazy to think of what the UX of that would be 

In [29]:
def runLLM(node_id, schema_dictionary):
    '''
        put a funciton that runs the LLM here
    '''
    #for testing just return a string to check that schema's working
    return "this is a test string"

    for node in schema_dictionary['nodes']:
        if node['id'] == node_id:
            prompt = ''
            for key in node['data'].keys():
                prompt += node['data'][key]
                prompt += " \n"
            # run LLM on prompt, note that this output will need to be sent over web somehow
    return prompt

In [30]:
def runSchema(schema_dictionary):
    '''
        Take a schema and run the flow. 

        There are 2 things to track, what the current graph looks like and the stack of 
        nodes to run and their received prompts. Nodes on the stack have to check if they are a 
        root, or else they are not run yet. Root nodes on the stack are run in order, then
        targets are added to the stack with their associated prompt. If the target is already
        on the stack, simply give it the additional prompt.

        If there are nodes, and all of the nodes have a source, this means that there is a loop 
        somewhere in the graph. We still want to run the flow, because recursive nodes are a 
        selling point. In this case, if ALL the nodes have a source, pick the most recently 
        created node and run it, sending its prompts and adding the new targets to the stack. We
        do not update the graph now, instead simply running the node, then its targets in order.
        This will continue running iterations, until a stop button is pressed.

        The user will probably want to

        We should create a stack, a list of dictionaries that look like this:
        [{node_id: ____, sources_dict:{source_id:received_prompt = "" }}]
        This forms a stack. Sources dict may or may not have multiple source_ids in it. Queue up
        the next nodes to run based on what the targets of the node you just ran are. If
        a node has multiple sources when it's run, combine those sources into one prompt
        via concatenation. Received prompts are added first, with the LLMs actual text
        entered into it added last. We may have to do prompt engineering to make sure
        the LLM answers the prompt inputted into the box and not questions received as context,
        but this is not preferred.
        For bonus points, add an option to use dfs or bfs
    '''
    ### Should listen here to see if user hit the pause/stop button, and if they did pause or stop the execution of the code

    roots = findRoots(schema_dictionary)
    
    #Base case: Check if schema dictionary has no roots
    if len(roots) == 0:
        if not schema_dictionary['edges']:
            sys.exit()
        #If schema dictionary has no roots, check if empty. If empty, exit with some success code
    
        #if schema dictionary has no roots and is not empty, there is a loop. Listen for a stop
        #signal from serverside, otherwise run schema dictionary in a loop 

    #Recursive case: Schema dictionary has roots. Get the outputs from the source node, make 
    #an updated schema dictionary where target nodes have the new outputs, and remove
    #the edges that have already been checked

    edge_ids_to_remove=[]
    nodes_to_send_outputs={}
    next_schema_dictionary=schema_dictionary.copy()
    for root in roots:
        for edge in next_schema_dictionary['edges']:
            if edge['source'] == root:
                edge_id = edge['id']
                edge_ids_to_remove.append(edge_id)
                nodes_to_send_outputs[edge['target']] = ""
                print(nodes_to_send_outputs)
                print(edge_ids_to_remove)
        output = runLLM(root, next_schema_dictionary)
        ### SEND OUTPUT TO CHATBOT TO OUTPUT HERE ####
        for node_id in nodes_to_send_outputs.keys():
            nodes_to_send_outputs[node_id] = output
        updated_prompts_dict = updateNodePrompts(nodes_to_send_outputs, schema_dictionary)
        next_schema_dictionary=removeEdgeIDs(edge_ids_to_remove, updated_prompts_dict)
    print(next_schema_dictionary)
    #runSchema(next_schema_dictionary)

In [31]:
runSchema(schema_dictionary)

{'3159559': ''}
['reactflow__edge-00001bottom-3159559top']
{'3159559': '', '3166128': ''}
['reactflow__edge-00001bottom-3159559top', 'reactflow__edge-00001right-3166128left']


KeyError: 'prompt'

In [15]:
schema_dictionary['edges']

[{'id': 'e1-2', 'source': '1', 'target': '2'},
 {'source': '00001',
  'sourceHandle': 'bottom',
  'target': '3159559',
  'targetHandle': 'top',
  'id': 'reactflow__edge-00001bottom-3159559top'},
 {'source': '00001',
  'sourceHandle': 'right',
  'target': '3166128',
  'targetHandle': 'left',
  'id': 'reactflow__edge-00001right-3166128left'},
 {'source': '3166128',
  'sourceHandle': 'bottom',
  'target': '3171323',
  'targetHandle': 'top',
  'id': 'reactflow__edge-3166128bottom-3171323top'},
 {'source': '3159559',
  'sourceHandle': 'right',
  'target': '3171323',
  'targetHandle': 'left',
  'id': 'reactflow__edge-3159559right-3171323left'}]